In [1]:
import json

from pymongo import MongoClient

In [2]:
with open('credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']

In [3]:
client = MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))
db = client.rais_anonymized
collection = db['fitbit']

In [4]:
#--------------------------------------------------------------------------------------------------------

#(i) Editing Type=Profile

#remove red fields as in the document
#1. full_name
#2. first_name
#3. last_name
#4. display_name
#5. display_name_setting
#6. username
#7. email_address
#8. timezone
#9. locale
#10. about_me
#
# add
#1.bmi

myquery = {"type": "Profile"}
newvalues = [
    {"$set": {"data.bmi":{ "$round" : [ { "$divide" :
                [ "$data.weight", { "$multiply" : [ "$data.height", "$data.height", 0.0001 ]} ]} ]} } },
    { "$unset": ["data.full_name", "data.first_name", "data.last_name", "data.display_name",
                          "data.display_name_setting", "data.username", "data.email_address",
                         "data.timezone", "data.locale", "data.about_me"]}
            ]
x = collection.update_many(myquery, newvalues)

In [5]:

# remove other fields
#1. id
#2. country
#3. state
#4. city
#5. height
#6. weight
#7. stride_length_walking
#8. stride_length_running

myquery = {"type": "Profile"}
newvalues = [{ "$unset": ["data.id", "data.country", "data.state", "data.city", "data.height", "data.weight", "data.stride_length_walking", "data.stride_length_running"]}]
x = collection.update_many(myquery, newvalues)

In [6]:

#Convert extreme bmi to ranges
myquery = {"type": "Profile", "data.gender": "MALE", "data.bmi":{"$gte":30}}
newvalues = [{"$set" : {"data.bmi":">=30"}}]
x = collection.update_many(myquery, newvalues)

In [7]:

myquery = {"type": "Profile", "data.gender": "FEMALE", "data.bmi":{"$gte":25}}
newvalues = [{"$set" : {"data.bmi":">=25"}}]
x = collection.update_many(myquery, newvalues)

In [8]:

myquery = {"type": "Profile", "data.bmi":{"$lt":19}}
newvalues = [{"$set" : {"data.bmi":"<19"}}]
x = collection.update_many(myquery, newvalues)


In [9]:

# convert date_of_birth to ages
from datetime import datetime
experiment_date = datetime(2021,5,24)
myquery = {"type": "Profile"}
newvalues = [{"$set" : {"data.age": {"$floor":[{ "$divide": [ { "$subtract":
            [experiment_date, {"$toDate":"$data.date_of_birth"} ] }, 1000*3600*24*365.2425] }] } }}]
x = collection.update_many(myquery, newvalues)

In [10]:

## convert age to ranges
myquery = {"type": "Profile", "data.age":{"$lt":30}}
newvalues = [{"$set" : {"data.age":"<30"}}]
x = collection.update_many(myquery, newvalues)

myquery = {"type": "Profile", "data.age":{"$gte":30}}
newvalues = [{"$set" : {"data.age":">=30"}}]
x = collection.update_many(myquery, newvalues)

In [11]:

#remove
#1. date_of_birth
#2. member_since

myquery = {"type": "Profile"}
newvalues = [{ "$unset": ["data.date_of_birth", "data.member_since"]}]
x = collection.update_many(myquery, newvalues)

In [12]:

#--------------------------------------------------------------------------------------------------------

#(ii) Editing Type=journal_entries

# remove
# 1. zone_offset

myquery = {"type": "journal_entries"}
newvalues = [
    { "$unset": ["data.zone_offset"]}
            ]

x = collection.update_many(myquery, newvalues)


In [13]:

#--------------------------------------------------------------------------------------------------------

#(iii) Editing Type=mindfulness_sessions

#remove time zone difference

myquery = {"type": "mindfulness_sessions"}
newvalues = [{"$set" : {"data.start_date_time":{"$substr":[ "$data.start_date_time", 0,
                                               { "$add": [ { "$strLenCP": "$data.start_date_time" }, -6 ] } ]} }},
            {"$set" : {"data.end_date_time":{"$substr":[ "$data.end_date_time", 0,
                                               { "$add": [ { "$strLenCP": "$data.end_date_time" }, -6 ] } ]} }}]

x = collection.update_many(myquery, newvalues)


In [14]:

#--------------------------------------------------------------------------------------------------------

#(iv) Editing Type=badge

#remove a single GOAL_BASED_WEIGHT_LOSS_ACHIEVEMENT

myquery = {"type":"badge","data.encodedId":'22984Z'}
x = collection.delete_many(myquery)

In [15]:

#add
#1. badge

# myquery = {"type":"badge"}
# newvalues = [{"$set" : {"data.badge":{"$concat":[ "$data.badgeType","=", { "$toString": "$data.value" } ]} }}]
# x = collection.update_many(myquery, newvalues)

In [16]:

#remove
#1. encodedId
#2. timesAchieved
#3. name
#4. shortName
#5. earnedMessage
#6. description
#7. category
#8. shareText

myquery = {"type": "badge"}
newvalues = [{ "$unset": ["data.encodedId", "data.timesAchieved", "data.name", "data.shortName",
				"data.earnedMessage", "data.description", "data.category", "data.shareText"]}]
x = collection.update_many(myquery, newvalues)

In [17]:

#--------------------------------------------------------------------------------------------------------

#(v) Editing Type=execise
# Example: 629718672c3a162f67928676

ACTIVITIES = {
    90013: "Walk",
    15000: "Sport",
    3001: "Aerobic Workout",
    52000: "Yoga/Pilates",
    90024: "Swim",
    90001: "Bike",
    20047: "Elliptical",
    2131: "Weights",
    55001: "Spinning",
    1071: "Bike",
    90009: "Run",
    20049: "Treadmill",
    53000: "Yoga/Pilates",
    55002: "Martial Arts",
    2040: "Circuit Training",
    2065: "Stairclimber",
    3000: "Workout",
    90012: "Hike",
    12339646: "Run",
    12350445: "Walk",
    23418750: "Swim",
    55003: "Bootcamp",
    15430: "Martial Arts",
    20057: "Interval Workout",
    15675: "Tennis",
    61980497: "Workout"
}


In [18]:
for activity_id, activity_name in ACTIVITIES.items():
    myquery = {"type": "exercise", "data.activityTypeId": activity_id}
    newvalue = [{"$set": {"data.activityName": activity_name}}]
    x = collection.update_many(myquery, newvalue)
    print(x.raw_result)

{'n': 3794, 'nModified': 740, 'ok': 1.0, 'updatedExisting': True}
{'n': 211, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}
{'n': 182, 'nModified': 4, 'ok': 1.0, 'updatedExisting': True}
{'n': 5, 'nModified': 5, 'ok': 1.0, 'updatedExisting': True}
{'n': 71, 'nModified': 22, 'ok': 1.0, 'updatedExisting': True}
{'n': 114, 'nModified': 7, 'ok': 1.0, 'updatedExisting': True}
{'n': 16, 'nModified': 2, 'ok': 1.0, 'updatedExisting': True}
{'n': 64, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}
{'n': 11, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}
{'n': 183, 'nModified': 183, 'ok': 1.0, 'updatedExisting': True}
{'n': 258, 'nModified': 46, 'ok': 1.0, 'updatedExisting': True}
{'n': 38, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}
{'n': 54, 'nModified': 54, 'ok': 1.0, 'updatedExisting': True}
{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}
{'n': 43, 'nModified': 21, 'ok': 1.0, 'updatedExisting': True}
{'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': Tr

In [34]:

#--------------------------------------------------------------------------------------------------------
#
#remove data outside of the experiment duration
#

from datetime import datetime
from_date = datetime(2021,5,24)
to_date = datetime(2022,1,22)

In [35]:

#1.log_time
#convert log_time to date
myquery = {"data.log_time":{"$exists": "true"}}
newvalues = [{"$set" : {"data.testdate":{"$toDate":"$data.log_time"}}}]
x = collection.update_many(myquery, newvalues)

KeyboardInterrupt: 

In [ ]:

#delete entries outside of experiment dates
myquery = {"$or": [
      {"data.log_time":{"$exists": "true"}, "data.testdate": {"$lt":from_date} },
      { "data.log_time":{"$exists": "true"}, "data.testdate": {"$gt":to_date } }
   ]}
x = collection.delete_many(myquery)

In [ ]:

#delete the previously added date fields
myquery = {"data.log_time":{"$exists": "true"}}
newvalues = [{"$unset" : ["data.testdate"]}]
x = collection.update_many(myquery, newvalues)

In [ ]:

#2.dateTime
#convert datetime to date
myquery = {"data.dateTime":{"$exists": "true"}}

newvalues = [{"$set" : {"data.testdate":{"$toDate":"$data.dateTime"}}}]
x = collection.update_many(myquery, newvalues)

In [ ]:

#delete entries outside of experiment dates
myquery = {"$or": [
      {"data.dateTime":{"$exists": "true"}, "data.testdate": {"$lt":from_date} },
      { "data.dateTime":{"$exists": "true"}, "data.testdate": {"$gt":to_date } }
   ]}
x = collection.delete_many(myquery)

In [ ]:

#delete the previously added date fields
myquery = {"data.dateTime":{"$exists": "true"}}
newvalues = [{"$unset" : ["data.testdate"]}]
x = collection.update_many(myquery, newvalues)

In [ ]:

#3.startTime
#convert startTime to date
myquery = {"data.startTime":{"$exists": "true"}}

newvalues = [{"$set" : {"data.testdate":{"$toDate":"$data.startTime"}}}]
x = collection.update_many(myquery, newvalues)

In [ ]:

#delete entries outside of experiment dates
myquery = {"$or": [
      {"data.startTime":{"$exists": "true"}, "data.testdate": {"$lt":from_date} },
      { "data.startTime":{"$exists": "true"}, "data.testdate": {"$gt":to_date } }
   ]}
x = collection.delete_many(myquery)

In [ ]:

#delete the previously added date fields
myquery = {"data.startTime":{"$exists": "true"}}
newvalues = [{"$unset" : ["data.testdate"]}]
x = collection.update_many(myquery, newvalues)

In [ ]:

#4.timestamp
#convert timestamp to date
#4.1. the milliseconds entries
myquery = {"data.timestamp":{"$exists": "true"}, "data.timestamp":{"$type":[16]}}
newvalues = [{"$set" : {"data.testdate":{"$toDate": { "$multiply": [{"$toDouble":"$data.timestamp"},1000 ] }}}}]

x = collection.update_many(myquery, newvalues)

In [ ]:

#4.2 the string entries
myquery = {"data.timestamp":{"$exists": "true"}, "data.timestamp":{"$type":[2]}}
newvalues = [{"$set" : {"data.testdate":{"$toDate":"$data.timestamp" }}}]

x = collection.update_many(myquery, newvalues)

In [ ]:

#delete entries outside of experiment dates
myquery = {"$or": [
      {"data.timestamp":{"$exists": "true"}, "data.testdate": {"$lt":from_date} },
      { "data.timestamp":{"$exists": "true"}, "data.testdate": {"$gt":to_date } }
   ]}
x = collection.delete_many(myquery)

In [ ]:

#delete the previously added date fields
myquery = {"data.timestamp":{"$exists": "true"}}
newvalues = [{"$unset" : ["data.testdate"]}]
x = collection.update_many(myquery, newvalues)

In [ ]:

#--------------------------------------------------------------------------------------------------------

#modify a duplicate entry

from bson.objectid import ObjectId
myquery = {"type": "Profile", "id":ObjectId("621e367e67b776a24087d75d")}
newvalues = [{ "$unset": ["data.bmi", "data.age"]}]

x = collection.update_many(myquery, newvalues)

In [ ]:

#4. reading_time
#convert reading_time to date
myquery = {"data.reading_time":{"$exists": "true"}}
newvalues = [{"$set" : {"data.testdate":{"$toDate":"$data.reading_time"}}}]
x = collection.update_many(myquery, newvalues)

In [ ]:

#delete entries outside of experiment dates
myquery = {"$or": [
      {"data.reading_time":{"$exists": "true"}, "data.testdate": {"$lt":from_date} },
      { "data.reading_time":{"$exists": "true"}, "data.testdate": {"$gt":to_date } }
   ]}
x = collection.delete_many(myquery)

In [ ]:

#delete the previously added date fields
myquery = {"data.reading_time":{"$exists": "true"}}
newvalues = [{"$unset" : ["data.testdate"]}]
x = collection.update_many(myquery, newvalues)

In [ ]:

#5. sleep_start
#convert sleep_start to date
myquery = {"data.sleep_start":{"$exists": "true"}}
newvalues = [{"$set" : {"data.testdate":{"$toDate":"$data.sleep_start"}}}]
x = collection.update_many(myquery, newvalues)

In [ ]:

#delete entries outside of experiment dates
myquery = {"$or": [
      {"data.sleep_start":{"$exists": "true"}, "data.testdate": {"$lt":from_date} },
      { "data.sleep_start":{"$exists": "true"}, "data.testdate": {"$gt":to_date } }
   ]}
x = collection.delete_many(myquery)

In [ ]:

#delete the previously added date fields
myquery = {"data.sleep_start":{"$exists": "true"}}
newvalues = [{"$unset" : ["data.testdate"]}]
x = collection.update_many(myquery, newvalues)

In [ ]:

#6. recorded_time
#convert recorded_time to date
myquery = {"data.recorded_time":{"$exists": "true"}}
newvalues = [{"$set" : {"data.testdate":{"$toDate":"$data.recorded_time"}}}]
x = collection.update_many(myquery, newvalues)

In [ ]:
#delete entries outside of experiment dates
myquery = {"$or": [
      {"data.recorded_time":{"$exists": "true"}, "data.testdate": {"$lt":from_date} },
      { "data.recorded_time":{"$exists": "true"}, "data.testdate": {"$gt":to_date } }
   ]}
x = collection.delete_many(myquery)

In [ ]:
#delete the previously added date fields
myquery = {"data.recorded_time":{"$exists": "true"}}
newvalues = [{"$unset" : ["data.testdate"]}]
x = collection.update_many(myquery, newvalues)

In [ ]:
#7. DATE
#convert DATE to date
myquery = {"data.DATE":{"$exists": "true"}}
newvalues = [{"$set" : {"data.testdate":{"$toDate":"$data.DATE"}}}]
x = collection.update_many(myquery, newvalues)

In [ ]:
#delete entries outside of experiment dates
myquery = {"$or": [
      {"data.DATE":{"$exists": "true"}, "data.testdate": {"$lt":from_date} },
      { "data.DATE":{"$exists": "true"}, "data.testdate": {"$gt":to_date } }
   ]}
x = collection.delete_many(myquery)

In [ ]:
#delete the previously added date fields
myquery = {"data.DATE":{"$exists": "true"}}
newvalues = [{"$unset" : ["data.testdate"]}]
x = collection.update_many(myquery, newvalues)

In [ ]:
#8. date
#convert date to date
myquery = {"data.date":{"$exists": "true"}}
newvalues = [{"$set" : {"data.testdate":{"$toDate":"$data.date"}}}]
x = collection.update_many(myquery, newvalues)

In [ ]:
#delete entries outside of experiment dates
myquery = {"$or": [
      {"data.date":{"$exists": "true"}, "data.testdate": {"$lt":from_date} },
      { "data.date":{"$exists": "true"}, "data.testdate": {"$gt":to_date } }
   ]}
x = collection.delete_many(myquery)

In [ ]:
#delete the previously added date fields
myquery = {"data.date":{"$exists": "true"}}
newvalues = [{"$unset" : ["data.testdate"]}]
x = collection.update_many(myquery, newvalues)

In [ ]:
#9. start_date_time
#convert start_date_time to date
myquery = {"data.start_date_time":{"$exists": "true"}}
newvalues = [{"$set" : {"data.testdate":{"$toDate":"$data.start_date_time"}}}]
x = collection.update_many(myquery, newvalues)

In [ ]:
#delete entries outside of experiment dates
myquery = {"$or": [
      {"data.start_date_time":{"$exists": "true"}, "data.testdate": {"$lt":from_date} },
      { "data.start_date_time":{"$exists": "true"}, "data.testdate": {"$gt":to_date } }
   ]}
x = collection.delete_many(myquery)

In [ ]:
#delete the previously added date fields
myquery = {"data.start_date_time":{"$exists": "true"}}
newvalues = [{"$unset" : ["data.testdate"]}]
x = collection.update_many(myquery, newvalues)

In [ ]:
# clean unnecessary types
#1. swim_lengths_data
#2. height
myquery = {"type":"swim_lengths_data"}
x = collection.delete_many(myquery)
myquery = {"type":"height"}
x = collection.delete_many(myquery)
#--------------------------------------------------------------------------------------------------------